In [155]:
import pandas as pd
import numpy as np


from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [172]:
movies = pd.read_csv('movies.csv')
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [173]:
movies.drop(['genres'], axis=1, inplace=True)

In [184]:
ratings = pd.read_csv('ratings.csv')
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [186]:
ratings.drop(['timestamp'], axis=1, inplace=True)

In [189]:
movies.head(10)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [190]:
ratings.head(10)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
5,1,70,3.0
6,1,101,5.0
7,1,110,4.0
8,1,151,5.0
9,1,157,5.0


In [197]:
user_item_matrix = ratings.pivot(index='movieId', columns='userId', values='rating')

In [198]:
user_item_matrix.shape

(9724, 610)

In [202]:
user_item_matrix.fillna(0,inplace=True)
user_itam_matrix.head()

userId,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,3.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [203]:
user_itam_matrix.shape

(2121, 378)

In [204]:
user_votes = ratings.groupby('userId')['rating'].agg('count')

movies_votes = ratings.groupby('movieId')['rating'].agg('count')


In [205]:
user_mask = user_votes[user_votes>50].index
movie_mask = movies_votes[movies_votes > 10].index

In [206]:
user_mask[0:20]

Int64Index([1, 4, 6, 7, 10, 11, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28,
            29, 32],
           dtype='int64', name='userId')

In [207]:
user_itam_matrix = user_item_matrix.loc[movie_mask,:]
user_itam_matrix = user_itam_matrix.loc[:, user_mask]

In [208]:
user_itam_matrix.shape

(2121, 378)

In [209]:
csr_data = csr_matrix(user_itam_matrix.values)

In [210]:
print(csr_data[:2, :5])

  (0, 0)	4.0
  (0, 3)	4.5
  (1, 2)	4.0


In [211]:
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
user_item_matrix.head()

,movieId,1,2,3,4,5,6,7,8,9,...,601,602,603,604,605,606,607,608,609,610
0,1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
1,2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
2,3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [212]:
knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 20, n_jobs = -1)


knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [213]:
recommendations = 10
search_word = 'Matrix'

In [214]:
movie_search = movies[movies['title'].str.contains(search_word)]
movie_search

,movieId,title
1939,2571,"Matrix, The (1999)"
4351,6365,"Matrix Reloaded, The (2003)"
4639,6934,"Matrix Revolutions, The (2003)"


In [215]:
movie_id = movie_search.iloc[0]['movieId']
 
movie_id = user_item_matrix[user_item_matrix['movieId'] == movie_id].index[0]
movie_id

1938

In [216]:
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors = recommendations + 1)


In [217]:
indices

array([[1938, 1995, 2022, 1891, 1887, 1993, 2014, 1937, 1911, 1989, 1868]])

In [218]:
distances

array([[0.        , 0.43860655, 0.48406401, 0.49128423, 0.49644158,
        0.50121794, 0.50670203, 0.50727252, 0.51616527, 0.51771662,
        0.51830934]])

In [219]:
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()
 

indices_distances = list(zip(indices_list, distances_list))
print(type(indices_distances[0]))
 
print(indices_distances[:3])

<class 'tuple'>
[(1938, 0.0), (1995, 0.4386065478686805), (2022, 0.484064005002772)]


In [220]:

indices_distances_sorted = sorted(indices_distances, key = lambda x: x[1], reverse = False)
indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

[(1995, 0.4386065478686805),
 (2022, 0.484064005002772),
 (1891, 0.4912842287278397),
 (1887, 0.4964415761945702),
 (1993, 0.501217939469273),
 (2014, 0.5067020305924875),
 (1937, 0.5072725163234262),
 (1911, 0.5161652705512947),
 (1989, 0.5177166189846252),
 (1868, 0.5183093379708843)]

In [221]:
# создаем пустой список, в который будем помещать название фильма и расстояние до него
recom_list = []

# теперь в цикле будем поочередно проходить по кортежам
for ind_dist in indices_distances_sorted:

    # искать movieId в матрице предпочтений
    matrix_movie_id = user_item_matrix.iloc[ind_dist[0]]['movieId']

    # выяснять индекс этого фильма в датафрейме movies
    id = movies[movies['movieId'] == matrix_movie_id].index

    # брать название фильма и расстояние до него
    title = movies.iloc[id]['title'].values[0]
    dist = ind_dist[1]

    # помещать каждую пару в питоновский словарь
    # который, в свою очередь, станет элементом списка recom_list
    recom_list.append({'Title' : title, 'Distance' : dist})

In [222]:
recom_list[0]

{'Title': 'Tarantula (1955)', 'Distance': 0.4386065478686805}

In [223]:
recom_df = pd.DataFrame(recom_list, index = range(1, recommendations + 1))
recom_df

,Title,Distance
1,Tarantula (1955),0.438607
2,"Boys, The (1998)",0.484064
3,Children of the Corn II: The Final Sacrifice (...,0.491284
4,"Long Goodbye, The (1973)",0.496442
5,"Wolf Man, The (1941)",0.501218
6,"Red Violin, The (Violon rouge, Le) (1998)",0.506702
7,"Walk on the Moon, A (1999)",0.507273
8,Beyond the Poseidon Adventure (1979),0.516165
9,Dracula (1931),0.517717
10,"Day of the Beast, The (Día de la Bestia, El) (...",0.518309
